In [1]:
!pip install simpletransformers
!pip install transformers==3.5.1
import pandas as pd

## Process data, assign labels
### Run this before running any training scripts

In [2]:
labeled_df = pd.read_csv('/content/drive/My Drive/NLP/data_final.csv')
#labeled_df = pd.read_csv('/content/drive/My Drive/NLP/fewshot_winogrande_10.csv')
#labeled_df = pd.read_csv('/content/drive/My Drive/NLP/fewshot_wsc_10.csv')
gap_df = pd.read_csv('/content/drive/My Drive/NLP/gap_coreference/gap-test.tsv', sep='\t')
win_f = '/content/drive/My Drive/NLP/winogrande_1.1/train_m.jsonl'
wsc_df = pd.read_csv('/content/drive/My Drive/NLP/wsc/wsc.csv')

In [3]:
import pandas as pd
import ast

def get_answer_index(ans_list):
    cand_list = ast.literal_eval(ans_list[0])
    answer = ans_list[1]
    if answer in cand_list:
        return cand_list.index(answer)
    else:
      print("Answer not found in sentence; will be dropped from dataset")
      return -1

start_size = len(labeled_df)
labeled_df['Num candidates'] = labeled_df['Possible answers'].apply(lambda x: len(ast.literal_eval(x)))
labeled_df = labeled_df[labeled_df['Num candidates'] > 1]
labeled_df['Answer index'] = labeled_df[['Possible answers', 'Answer']].apply(get_answer_index, axis=1)
labeled_df = labeled_df[labeled_df['Answer index'] != -1]
labeled_df['Answer index'] = labeled_df['Answer index'].apply(lambda x: int(x) if x == 0 or x == 1 else 2)
labeled_df.rename(columns={'Answer index': 'Label'}, inplace=True)
print("Using {} samples out of {}...".format(len(labeled_df), start_size))
#labeled_df.to_csv('/content/drive/My Drive/NLP/data_labeled.csv')
labeled_df

Answer not found in sentence; will be dropped from dataset
Using 6220 samples out of 6330...


,Unnamed: 0,Unnamed: 0.1,Text,Pronoun,Pronoun-offset,Answer,Answer-offset,Possible answers,Num candidates,Label
0,0,0,"After the war, Herbert attended the University...",he,67,Herbert,67,"['Herbert', 'Beverly']",2,0
1,1,1,Herbert was appalled to learn of senator McCar...,he,138,Herbert,138,"['Herbert', 'McCarthy']",2,0
2,2,2,Herbert has attracted a sometimes fanatical fa...,he,96,Herbert,96,"['Herbert', 'Herbert']",2,0
3,3,3,"In 1887, after the death of his brother and a ...",he,119,Julian,119,"['Norris', 'Julian', 'Zola']",3,1
4,4,4,"The publication of Buechner's third novel, The...",he,125,Buechner,125,"['Buechner', 'Buechner']",2,0
...,...,...,...,...,...,...,...,...,...,...
6325,75,75,Then slice the cabbage into small pieces and w...,it,51,cabbage,16,"['cabbage', 'pieces']",2,0
6326,76,76,Add a little bit of salt and give it a rough mix.,it,8,salt,21,"['bit', 'salt', 'mix']",3,1
6327,77,77,Now add required amount of mayonnaise to the b...,it,68,mayonnaise,28,"['amount', 'mayonnaise', 'bowl']",3,1
6328,78,78,After the dry is all mixed up add the eggs and...,it,64,dry,11,"['dry', 'eggs', 'oil']",3,0


##BERT implementation
###   Multiclass classification:
* Use all samples that have at least two answer candidates
*     Label is the index of the correct answer in the list of candidates
*     Label = 0 (A), 1 (B), or 2 (Neither)





#### Train and test on our dataset

In [9]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

alldata_df = labeled_df

# Shuffle
alldata_df = alldata_df.sample(frac=1)

# Create 60%, 20%, 20% split of train, eval, test 
train_df = alldata_df[:int(len(alldata_df)*0.8)]
test_df = alldata_df[int(len(alldata_df)*0.8):]
train_df = train_df[:int(len(train_df)*0.75)]
eval_df = train_df[int(len(train_df)*0.75):]

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
train_dict = {'text': train_df['Text'], 'label': train_df['Label']}
train_df_final = pd.DataFrame(train_dict)

eval_dict = {'text': eval_df['Text'], 'label': eval_df['Label']}
eval_df_final = pd.DataFrame(eval_dict)

# Create a ClassificationModel
#model = ClassificationModel('bert', 'bert-base-cased', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 20})
#model = ClassificationModel('roberta', 'roberta-base', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 20})
model = ClassificationModel('albert', 'albert-base-v2', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 20})
print(train_df_final)
# Train the model
model.train_model(train_df_final, output_dir='/content/drive/My Drive/NLP/models/albert-base-ourdataset')

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df_final)


INFO:filelock:Lock 139831963680048 acquired on /root/.cache/torch/transformers/e48be00f755a5f765e36a32885e8d6a573081df3321c9e19428d12abadf7dba2.b8f28145885741cf994c0e8a97b724f6c974460c297002145e48e511d2496e88.lock


INFO:filelock:Lock 139831963680048 released on /root/.cache/torch/transformers/e48be00f755a5f765e36a32885e8d6a573081df3321c9e19428d12abadf7dba2.b8f28145885741cf994c0e8a97b724f6c974460c297002145e48e511d2496e88.lock
INFO:filelock:Lock 139831965704320 acquired on /root/.cache/torch/transformers/bf1986d976e9a8320cbd3a0597e610bf299d639ce31b7ca581cbf54be3aaa6d3.d6d54047dfe6ae844e3bf6e7a7d0aff71cb598d3df019361e076ba7639b1da9b.lock


INFO:filelock:Lock 139831965704320 released on /root/.cache/torch/transformers/bf1986d976e9a8320cbd3a0597e610bf299d639ce31b7ca581cbf54be3aaa6d3.d6d54047dfe6ae844e3bf6e7a7d0aff71cb598d3df019361e076ba7639b1da9b.lock


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

INFO:filelock:Lock 139831816716072 released on /root/.cache/torch/transformers/10be6ce6d3508f1fdce98a57a574283b47c055228c1235f8686f039287ff8174.d6110e25022b713452eb83d5bfa8ae64530995a93d8e694fe52e05aa85dd3a7d.lock
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:380: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


...

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f2d21f0ea20>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process


INFO:simpletransformers.classification.classification_model: Training of albert model complete. Saved to /content/drive/My Drive/NLP/models/albert-base-ourdataset.


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:901: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 1.0, 'eval_loss': 8.787348544237286e-06}


In [10]:
from sklearn.metrics import confusion_matrix, classification_report
# Predict and get metrics
predictions, raw_outputs = model.predict(test_df['Text'].tolist())
print("\n{}".format(confusion_matrix(test_df['Label'].tolist(), predictions)))
print("\n{}".format(classification_report(test_df['Label'].tolist(), predictions, digits=4)))

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.




[[745  92   8]
 [121 190   9]
 [ 27  36  16]]

              precision    recall  f1-score   support

           0     0.8343    0.8817    0.8573       845
           1     0.5975    0.5938    0.5956       320
           2     0.4848    0.2025    0.2857        79

    accuracy                         0.7645      1244
   macro avg     0.6389    0.5593    0.5795      1244
weighted avg     0.7512    0.7645    0.7537      1244



###Train on our dataset, test on GAP Coreference

In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

alldata_df = labeled_df

# Shuffle
alldata_df = alldata_df.sample(frac=1)

# Create 80% train, 20% eval 
train_df = alldata_df[:int(len(alldata_df)*0.8)]
eval_df = alldata_df[int(len(alldata_df)*0.8):]

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
train_dict = {'text': train_df['Text'], 'label': train_df['Label']}
train_df_final = pd.DataFrame(train_dict)

eval_dict = {'text': eval_df['Text'], 'label': eval_df['Label']}
eval_df_final = pd.DataFrame(eval_dict)

# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-cased', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 10})
print(train_df_final)
# Train the model
model.train_model(train_df_final, output_dir='/content/drive/My Drive/NLP/models/bert-base-cased-gap')

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df_final)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

                                                   text  label
5396  The president of the Juilliard School consulte...      0
3904  "Piano": Lucy annoys Schroeder while he is pla...      1
2766  Authorization for such a promotion "with the a...      0
12    Before the March on Rome, De even went so far ...      2
5978  Ralph said he and Sting were not paid for thei...      0
...                                                 ...    ...
4385  Evelyn was educated at home until the age of 1...      0
1685  Rowling has lived a "rags to riches" life in w...      0
4453  When Baudelaire returned from Belgium after hi...      1
2536  In 1996, Peter announced that he and Sharp wou...      0
2291  Rahman is the main judge and he is accompanied...      0

[4976 rows x 2 columns]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to /content/drive/My Drive/NLP/models/bert-base-cased-gap.
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:896: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.6414865360038053, 'eval_loss': 1.5143181399368517}


In [ ]:
def assign_label(coref_list):
    if coref_list[0] == True:
        return 0
    elif coref_list[1] == True:
        return 1
    else:
        return 2

gap_df['Label'] = gap_df[['A-coref', 'B-coref']].apply(assign_label, axis=1)
gap_df

,ID,Text,Pronoun,Pronoun-offset,A,A-offset,A-coref,B,B-offset,B-coref,URL,Label
0,test-1,Upon their acceptance into the Kontinental Hoc...,His,383,Bob Suter,352,False,Dehner,366,True,http://en.wikipedia.org/wiki/Jeremy_Dehner,1
1,test-2,"Between the years 1979-1981, River won four lo...",him,430,Alonso,353,True,Alfredo Di St*fano,390,False,http://en.wikipedia.org/wiki/Norberto_Alonso,0
2,test-3,Though his emigration from the country has aff...,He,312,Ali Aladhadh,256,True,Saddam,295,False,http://en.wikipedia.org/wiki/Aladhadh,0
3,test-4,"At the trial, Pisciotta said: ``Those who have...",his,526,Alliata,377,False,Pisciotta,536,True,http://en.wikipedia.org/wiki/Gaspare_Pisciotta,1
4,test-5,It is about a pair of United States Navy shore...,his,406,Eddie,421,True,Rock Reilly,559,False,http://en.wikipedia.org/wiki/Chasers,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,test-1996,"The sole exception was Wimbledon, where she pl...",She,479,Goolagong Cawley,400,True,Peggy Michel,432,False,http://en.wikipedia.org/wiki/Evonne_Goolagong_...,0
1996,test-1997,"According to news reports, both Moore and Fily...",her,338,Esther Sheryl Wood,263,True,Barbara Morgan,404,False,http://en.wikipedia.org/wiki/Hastings_Arthur_Wise,0
1997,test-1998,"In June 2009, due to the popularity of the Sab...",She,328,Kayla,364,True,Natasha Henstridge,412,False,http://en.wikipedia.org/wiki/Raya_Meddine,0
1998,test-1999,She was delivered to the Norwegian passenger s...,she,305,Irma,255,True,Bergen,274,False,http://en.wikipedia.org/wiki/SS_Irma_(1905),0


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
# Predict and get metrics
predictions, raw_outputs = model.predict(gap_df['Text'])
print("\n{}".format(confusion_matrix(gap_df['Label'], predictions)))
print("\n{}".format(classification_report(gap_df['Label'], predictions, digits=4)))

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.




[[587 213 118]
 [459 252 144]
 [126  40  61]]

              precision    recall  f1-score   support

           0     0.5009    0.6394    0.5617       918
           1     0.4990    0.2947    0.3706       855
           2     0.1889    0.2687    0.2218       227

    accuracy                         0.4500      2000
   macro avg     0.3962    0.4010    0.3847      2000
weighted avg     0.4647    0.4500    0.4414      2000



###Train on our dataset, test on Winogrande

In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

alldata_df = labeled_df

# Shuffle
alldata_df = alldata_df.sample(frac=1)

# Create 80% train, 20% eval 
train_df = alldata_df[:int(len(alldata_df)*0.8)]
eval_df = alldata_df[int(len(alldata_df)*0.8):]

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
#train_data = [['Example sentence belonging to class 1', 1], ['Example sentence belonging to class 0', 0]]
train_dict = {'text': train_df['Text'], 'label': train_df['Label']}
train_df_final = pd.DataFrame(train_dict)
# train_df_final.reset_index(drop=True, inplace=True)

eval_dict = {'text': eval_df['Text'], 'label': eval_df['Label']}
eval_df_final = pd.DataFrame(eval_dict)

# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-cased', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 20})
#model = ClassificationModel('roberta', 'roberta-base', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 20})
#model = ClassificationModel('albert', 'albert-base-v2', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 20})

print(train_df_final)
# Train the model
model.train_model(train_df_final, output_dir='/content/drive/My Drive/NLP/models/test-bert-base-winogrande-fewshot')

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df_final)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

                                                   text  label
6226  The GPS and map helped me navigate home.  I go...      1
6227  The GPS and map helped me navigate home.  I go...      0
6223  He never comes to my home, but I always go to ...      1
6228  Emily looked up and saw Patricia racing by ove...      1
6224  Kyle doesn't wear leg warmers to bed, while Lo...      1
6221  Ian volunteered to eat Dennis's menudo after a...      0
6220  Ian volunteered to eat Dennis's menudo after a...      1
6225  Kyle doesn't wear leg warmers to bed, while Lo...      0


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:380: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to /content/drive/My Drive/NLP/models/test-bert-base-winogrande-fewshot.
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:901: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:simpletransformers.classification.classification_model:{'mcc': 0.0, 'eval_loss': 1.6565513610839844}


In [ ]:
import json
import random

win_text = []
win_labels = []
cnt = 0
with open(win_f, 'r') as f:
    for line in f:
        sample = json.loads(line)
        #pronoun = random.choice(['he', 'she'])
        #win_text.append(sample['sentence'].replace('_', pronoun))
        if cnt >= 10:
            win_text.append(sample['sentence'])
            win_labels.append(int(sample['answer']))
        cnt += 1

win_labels = [label-1 for label in win_labels]
print(win_text[0], win_labels[0])
print(len(win_text), len(win_labels))

The treasury workers took the gold bars off of the trolley and stacked them in the safe until the _ was empty. 1
2548 2548


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
# Predict and get metrics
predictions, raw_outputs = model.predict(win_text)
print("\n{}".format(confusion_matrix(win_labels, predictions)))
print("\n{}".format(classification_report(win_labels, predictions, digits=4)))

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.




[[   0 1274]
 [   0 1274]]

              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1274
           1     0.5000    1.0000    0.6667      1274

    accuracy                         0.5000      2548
   macro avg     0.2500    0.5000    0.3333      2548
weighted avg     0.2500    0.5000    0.3333      2548



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Train on our dataset, test on WSC

In [4]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging

alldata_df = labeled_df

# Shuffle
alldata_df = alldata_df.sample(frac=1)

# Create 80% train, 20% eval 
train_df = alldata_df[:int(len(alldata_df)*0.8)]
eval_df = alldata_df[int(len(alldata_df)*0.8):]

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
train_dict = {'text': train_df['Text'], 'label': train_df['Label']}
train_df_final = pd.DataFrame(train_dict)

eval_dict = {'text': eval_df['Text'], 'label': eval_df['Label']}
eval_df_final = pd.DataFrame(eval_dict)

# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-cased', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 20})
#model = ClassificationModel('roberta', 'roberta-base', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 20})
#model = ClassificationModel('albert', 'albert-base-v2', num_labels=3, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 20})

print(train_df_final)
# Train the model
model.train_model(train_df_final, output_dir='/content/drive/My Drive/NLP/models/bert-base-cased-wsc')

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df_final)


INFO:filelock:Lock 139831979472544 acquired on /root/.cache/torch/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655.lock


INFO:filelock:Lock 139831979472544 released on /root/.cache/torch/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.0d87139f53a477d9f900f8a9020c367863079014bdaf2aa713f4b64cf1782655.lock
INFO:filelock:Lock 139831979473160 acquired on /root/.cache/torch/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock


INFO:filelock:Lock 139831979473160 released on /root/.cache/torch/transformers/092cc582560fc3833e556b3f833695c26343cb54b7e88cd02d40821462a74999.1f48cab6c959fc6c360d22bea39d06959e90f5b002e77e836d2da45464875cda.lock


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

INFO:filelock:Lock 139831979474616 released on /root/.cache/torch/transformers/6508e60ab3c1200bffa26c95f4b58ac6b6d95fba4db1f195f632fa3cd7bc64cc.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791.lock



                                                   text  label
6132  Barty can really ride a horse, and he triumphs...      0
6136  Jones, an actor as divided as Two-Face, who is...      0
620   In 1815, Nicholas arrived in France, where he ...      0
3212  The exact number of destroyed manuscripts is u...      0
1522  When Stevens is diagnosed with stage IV metast...      0
...                                                 ...    ...
2754  Like Logan, Kirsch is critical of Omeros, whic...      2
3100  calan's brother Osman became a PKK commander u...      1
1739  Sanchez stated before the start of the 2016 se...      0
5270  Gilda was the top-billed actress, but Variety ...      0
5235  Bogart thought that the Warners wardrobe depar...      0

[4976 rows x 2 columns]


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:380: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to /content/drive/My Drive/NLP/models/bert-base-cased-wsc.
/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:901: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."
INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.6251918048458065, 'eval_loss': 1.895770060665261}


In [5]:
wsc_text = list(wsc_df['Sentence'])
wsc_labels = list(wsc_df['Label'])
# wsc_text = list(wsc_df['Sentence'])[10:]
# wsc_labels = list(wsc_df['Label'])[10:]
print(wsc_text[0], wsc_labels[0])

The city councilmen refused the demonstrators a permit because they feared violence. 0


In [6]:
from sklearn.metrics import confusion_matrix, classification_report
# Predict and get metrics
predictions, raw_outputs = model.predict(wsc_text)
print("\n{}".format(confusion_matrix(wsc_labels, predictions)))
print("\n{}".format(classification_report(wsc_labels, predictions, digits=4)))

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.




[[107  36]
 [104  38]]

              precision    recall  f1-score   support

           0     0.5071    0.7483    0.6045       143
           1     0.5135    0.2676    0.3519       142

    accuracy                         0.5088       285
   macro avg     0.5103    0.5079    0.4782       285
weighted avg     0.5103    0.5088    0.4786       285



In [ ]:
model.args

ClassificationArgs(adam_epsilon=1e-08, best_model_dir='outputs/best_model', cache_dir='cache_dir/', config={}, custom_layer_parameters=[], custom_parameter_groups=[], dataloader_num_workers=1, do_lower_case=False, dynamic_quantize=False, early_stopping_consider_epochs=False, early_stopping_delta=0, early_stopping_metric='eval_loss', early_stopping_metric_minimize=True, early_stopping_patience=3, encoding=None, eval_batch_size=8, evaluate_during_training=False, evaluate_during_training_silent=True, evaluate_during_training_steps=2000, evaluate_during_training_verbose=False, evaluate_each_epoch=True, fp16=True, gradient_accumulation_steps=1, learning_rate=4e-05, local_rank=-1, logging_steps=50, manual_seed=None, max_grad_norm=1.0, max_seq_length=128, model_name='albert-base-v2', model_type='albert', multiprocessing_chunksize=500, n_gpu=1, no_cache=False, no_save=False, not_saved_args=[], num_train_epochs=20, output_dir='outputs/', overwrite_output_dir=True, process_count=1, quantized_mod